In [7]:
import findspark
findspark.init()


In [8]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1 pyspark-shell'


In [9]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, udf
from pyspark.sql.types import StringType

import json
import datetime

from afinn import Afinn
spark = SparkSession.builder \
                    .master("local[3]") \
                    .config('spark.executor.instances', 3) \
                    .config("spark.sql.catalogImplementation", "hive")\
                    .appName('tweets') \
                    .getOrCreate()
ssc = StreamingContext(spark.sparkContext, 1)


Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [ ]:
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
tweetsDfRaw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test1") \
  .load()

In [ ]:
tweetsDf = tweetsDfRaw.selectExpr("CAST(value AS STRING) as tweet_info")

In [ ]:
def extract_tags(word: str):
    if word.lower().startswith("#"):
        return word
    else:
        return "nonTag"

In [29]:
def extract_tweet_text(text):
    tweet_info = json.loads(text)
    return tweet_info["tweet"]

def extract_timestamp(text):
    tweet_info = json.loads(text)
    created_at = tweet_info["created_at"]
    return datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y')

In [ ]:
afinn = Afinn()

In [ ]:

extract_tags_udf = udf(extract_tags, StringType())

resultDf = words.filter(words.word.isNotNull()).withColumn("tags", extract_tags_udf(words.word))

hashtagCountsDf = resultDf.filter(resultDf.tags != "nonTag")\
                            .groupBy("tags")\
                            .count()\
                            .orderBy("count", ascending=False)

In [ ]:
query = hashtagCountsDf.writeStream \
.outputMode("complete") \
.format("console")\
.trigger(processingTime='2 seconds') \
.option("truncate", "false")\
.start()\
.awaitTermination()
